In [1]:
import pandas as pd 
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
denver_data = pd.read_csv("../datasets/denver-data.csv") 
# Preview the first 5 lines of the loaded data 
denver_data.head(5)

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC
0,2.016377e+09,2.016380e+15,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,3193983.0,1707251.0,-104.809881,39.773188,5,521,montbello,1,0
1,2.018600e+10,2.018600e+16,2399,0,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,3201943.0,1711852.0,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0
2,2.016600e+10,2.016600e+16,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,2932 S JOSEPHINE ST,3152762.0,1667011.0,-104.957381,39.663490,3,314,wellshire,1,0
3,2.018723e+08,2.018720e+14,2399,0,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,3157162.0,1681320.0,-104.941440,39.702698,3,312,belcaro,1,0
4,2.017411e+09,2.017410e+15,2303,0,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,3153211.0,1686545.0,-104.955370,39.717107,3,311,cherry-creek,1,0


In [2]:
# Drop uneeded columns from dataset
data = denver_data.drop(['INCIDENT_ID', 'OFFENSE_ID', 'OFFENSE_CODE', 
                                    'OFFENSE_CODE_EXTENSION', 'OFFENSE_TYPE_ID',
                                    'OFFENSE_CATEGORY_ID', 'REPORTED_DATE',
                                    'GEO_X', 'GEO_Y', 'DISTRICT_ID', 'LAST_OCCURRENCE_DATE',
                                    'PRECINCT_ID', 'IS_CRIME', 'IS_TRAFFIC', 'GEO_LON',
                                    'GEO_LAT', 'NEIGHBORHOOD_ID', 'INCIDENT_ADDRESS'], axis=1)
data.head(5)

,FIRST_OCCURRENCE_DATE
0,6/15/2016 11:31:00 PM
1,10/11/2017 12:30:00 PM
2,3/4/2016 8:00:00 PM
3,1/30/2018 7:20:00 PM
4,6/22/2017 8:53:00 PM


In [3]:
# Splitting up date into Year, Month, Day, dataset has Month/Day/Year
date_values = data['FIRST_OCCURRENCE_DATE'].values

year = []
month = []
day = []
completeDay = []

for date in date_values:
    date = date.split() # Split on whitespace
    date = date[0] # Grab first portion which m/d/y
    completeDay.append(date)# Store the complete day to remove duplicates later
    date = date.split('/') # Split into m, d and y and turn into int
    month.append(int(date[0]))
    day.append(int(date[1]))
    year.append(int(date[2]))

data['year'] = year
data['month'] = month
data['day'] = day
data['completeDate'] = completeDay
data.head(5)

,FIRST_OCCURRENCE_DATE,year,month,day,completeDate
0,6/15/2016 11:31:00 PM,2016,6,15,6/15/2016
1,10/11/2017 12:30:00 PM,2017,10,11,10/11/2017
2,3/4/2016 8:00:00 PM,2016,3,4,3/4/2016
3,1/30/2018 7:20:00 PM,2018,1,30,1/30/2018
4,6/22/2017 8:53:00 PM,2017,6,22,6/22/2017


In [4]:
print('Total number of rows, including duplicates:', data.shape[0])

Total number of rows, including duplicates: 466840


In [5]:
# Dropping our duplicate location_name rows here
data.drop_duplicates(subset ="completeDate", inplace = True) 
print('Total number of rows, without duplicates:', data.shape[0])

Total number of rows, without duplicates: 1870


In [6]:
data.head(5)

,FIRST_OCCURRENCE_DATE,year,month,day,completeDate
0,6/15/2016 11:31:00 PM,2016,6,15,6/15/2016
1,10/11/2017 12:30:00 PM,2017,10,11,10/11/2017
2,3/4/2016 8:00:00 PM,2016,3,4,3/4/2016
3,1/30/2018 7:20:00 PM,2018,1,30,1/30/2018
4,6/22/2017 8:53:00 PM,2017,6,22,6/22/2017


In [7]:
# Drop uneeded columns again
data = data.drop(['FIRST_OCCURRENCE_DATE', 'completeDate'], axis=1)
data.head(5)

,year,month,day
0,2016,6,15
1,2017,10,11
2,2016,3,4
3,2018,1,30
4,2017,6,22


In [8]:
#trying to get the day of the week attribute
import datetime
calender = { 0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday' }
days = data['day'].values
months = data['month'].values
years = data['year'].values
days_of_the_week = []
for day, month, year in zip(days, months, years):
    day_of_week = calender[datetime.date(year, month, day).weekday()]
    days_of_the_week.append(day_of_week)

In [9]:
#adding day_of_the_week attribute to the data frame
data['day_of_week'] = days_of_the_week
data.head(5)

,year,month,day,day_of_week
0,2016,6,15,Wednesday
1,2017,10,11,Wednesday
2,2016,3,4,Friday
3,2018,1,30,Tuesday
4,2017,6,22,Thursday


In [10]:
#adding weekend attribute to dataframe
is_weekend = []
for day in days_of_the_week:
    if day == 'Saturday' or day == 'Sunday':
        is_weekend.append(True)
    else:
        is_weekend.append(False)
data['is_weekend'] = is_weekend
data.head(5)

,year,month,day,day_of_week,is_weekend
0,2016,6,15,Wednesday,False
1,2017,10,11,Wednesday,False
2,2016,3,4,Friday,False
3,2018,1,30,Tuesday,False
4,2017,6,22,Thursday,False


In [ ]:
#install the holidays needed; this library is included in requirements.txt

In [11]:
#setting up the holidays dictionary for colorado across the appropriate dates
import holidays
min_year_of_ds = data['year'].min()
max_year_of_ds = data['year'].max()
year_interval = [i for i in range(min_year_of_ds, max_year_of_ds+1, 1)]
co_holidays = holidays.US(years=year_interval, state='CO')

In [13]:
#working on the is-holiday attribute and holiday_name attribute
is_holiday = []
holiday_name = []
for day, month, year in zip(days, months, years):
    is_holiday.append(datetime.date(year, month, day) in co_holidays)
    if datetime.date(year, month, day) in co_holidays:
        holiday_name.append(co_holidays[datetime.date(year, month, day)])
    else:
        holiday_name.append(None)
data['is_holiday'] = is_holiday
data['holiday_name'] = holiday_name
data.head(5)

,year,month,day,day_of_week,is_weekend,is_holiday,holiday_name
0,2016,6,15,Wednesday,False,False,None
1,2017,10,11,Wednesday,False,False,None
2,2016,3,4,Friday,False,False,None
3,2018,1,30,Tuesday,False,False,None
4,2017,6,22,Thursday,False,False,None


In [14]:
#add date_key
import uuid
date_keys = []
for i in range(data.shape[0]):
    id = uuid.uuid4() 
    date_keys.append(id)

data['date_key'] = date_keys
data.head(5)

,year,month,day,day_of_week,is_weekend,is_holiday,holiday_name,date_key
0,2016,6,15,Wednesday,False,False,None,7b204d79-97d8-4eeb-8815-a64082e5f4f1
1,2017,10,11,Wednesday,False,False,None,8ae8bcc1-3af4-46cc-9d4e-c0bce6496a22
2,2016,3,4,Friday,False,False,None,6a0a1ef3-1ed6-4ef5-abad-2e4466ae33f1
3,2018,1,30,Tuesday,False,False,None,9fcc7225-793e-4dc8-9717-737149233431
4,2017,6,22,Thursday,False,False,None,25df993f-fe14-409a-bdb2-eb2f353a9f45


In [15]:
#rearrange columns of the dataframe
cols = data.columns.tolist()
new_cols = [cols[7]]
new_cols = new_cols + cols[0:7]
data = data[new_cols]
data.head()

,date_key,year,month,day,day_of_week,is_weekend,is_holiday,holiday_name
0,7b204d79-97d8-4eeb-8815-a64082e5f4f1,2016,6,15,Wednesday,False,False,None
1,8ae8bcc1-3af4-46cc-9d4e-c0bce6496a22,2017,10,11,Wednesday,False,False,None
2,6a0a1ef3-1ed6-4ef5-abad-2e4466ae33f1,2016,3,4,Friday,False,False,None
3,9fcc7225-793e-4dc8-9717-737149233431,2018,1,30,Tuesday,False,False,None
4,25df993f-fe14-409a-bdb2-eb2f353a9f45,2017,6,22,Thursday,False,False,None


In [16]:
# Finally here we convert the dataframe to a csv file to store in our repo
denver_date_csv = data.to_csv(r'./denver-date.csv', index = None, header=True)

In [ ]:
# End